## ●Notebookの内容

元データの読み込み、データの準備(整理)、準備後データの書き出し

# 1. パッケージのインポート・乱数固定

In [1]:
import pandas as pd #pandasパッケージをインポート
import numpy as np #numpyパッケージをインポート
import torch #ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn #「ニューラルネットワーク」モジュールの別名定義
import torch.nn.functional as F #「ニューラルネットワーク・活性化関数」モジュールの別名定義
import collections
import os
import pickle
import optuna
import torch.optim as optim

#乱数の固定
torch.manual_seed(123)# 1. 前準備

# 2. データの読み込み

## 2.1 最初に取得したデータの読み込み

In [2]:
#データの読み込み
#2017-2022のデータの読み込み
data_0_1 = pd.read_excel('./2017-2022.xlsx', header=None, index_col=None)
#2011-2016のデータの読み込み
data_0_2 = pd.read_excel('./2011-2016.xlsx', header=None, index_col=None)
#2005-2010のデータの読み込み
data_0_3 = pd.read_excel('./2005-2010.xlsx', header=None, index_col=None)
#1999-2004のデータの読み込み
data_0_4 = pd.read_excel('./1999-2004.xlsx', header=None, index_col=None)
#1993-1998のデータの読み込み
data_0_5 = pd.read_excel('./1993-1998.xlsx', header=None, index_col=None)
#1992のデータの読み込み
data_0_6 = pd.read_excel('./1992.xlsx', header=None, index_col=None)
#1987-1992のデータの読み込み
#data_0_6 = pd.read_excel('./1987-1992.xlsx', header=None, index_col=None)
#1983-1986のデータの読み込み
#data_0_7 = pd.read_excel('./1983-1986.xlsx', header=None, index_col=None)

In [3]:
data_0_1.head(2) #データフレームの要約の表示

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,2022/06/23 18:48:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,#A1101868000,2017-01-01 00:00:00,NaN,銘柄名,期日,ISINｺｰﾄﾞ,銘柄ｺｰﾄﾞ,始値,高値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)


## 2.2 追加で取得したデータの読み込み

In [4]:
#データの読み込み
#2017-2022の追加財務データの読み込み
zaimudata_0_1 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/2017-2022(財務).xlsx', header=None, index_col=None)
#2011-2016の追加財務データの読み込み
zaimudata_0_2 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/2011-2016(財務).xlsx', header=None, index_col=None)
#2005-2010の追加財務データの読み込み
zaimudata_0_3 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/2005-2010(財務).xlsx', header=None, index_col=None)
#1999-2004の追加財務データの読み込み
zaimudata_0_4 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/1999-2004(財務).xlsx', header=None, index_col=None)
#1993-1998の追加財務データの読み込み
zaimudata_0_5 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/1993-1998(財務).xlsx', header=None, index_col=None)
#1992の追加財務データの読み込み
zaimudata_0_6 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/1992(財務).xlsx', header=None, index_col=None)
#1992-2022の時価総額等データ
jikasougakudata_0_1 = pd.read_excel('/Users/nagashimahikaru/Documents/修士論文/使用データ/1992-2022(時価総額等).xlsx', header=None, index_col=None)

In [5]:
zaimudata_0_1.head(2) #データフレームの要約の表示

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,2022/09/23 12:48:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,#A1101851000,2017-01-01 00:00:00,NaN,銘柄名,期日,銘柄ｺｰﾄﾞ,流動比率,当座比率,固定比率,...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比


# 3. データの結合(読み込みファイル毎)

In [6]:
#上で読み込んだデータを結合(1992年以降のみ)
data_0 = pd.concat([data_0_1, data_0_2, data_0_3, data_0_4, data_0_5, data_0_6],axis=0,ignore_index=True)
zaimudata_0 = pd.concat([zaimudata_0_1, zaimudata_0_2, zaimudata_0_3, zaimudata_0_4, zaimudata_0_5, zaimudata_0_6],axis=0,ignore_index=True)

data_0 = data_0.rename(columns={4: 'Company', 5: 'Date', 7: 'Code'}) #特定の列の名前を変更
zaimudata_0 = zaimudata_0.rename(columns={4: 'Company', 5: 'Date', 6: 'Code'}) #特定の列の名前を変更
jikasougakudata_0_2_1 = jikasougakudata_0_1.rename(columns={4: 'Company', 5: 'Date',6: 'Code'}) #特定の列の名前を変更
jikasougakudata_0_2 = jikasougakudata_0_2_1.iloc[:,[4,5,6,7,8]]

display(data_0, zaimudata_0, jikasougakudata_0_2)

,0,1,2,3,Company,Date,6,Code,8,9,...,121,122,123,124,125,126,127,128,129,130
0,2022/06/23 18:48:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,#A1101868000,2017-01-01 00:00:00,NaN,銘柄名,期日,ISINｺｰﾄﾞ,銘柄ｺｰﾄﾞ,始値,高値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)
2,1332,#A1101851000,2022-06-01 00:00:00,NaN,極洋,2017/01,JP3257200000,1301,2734,2755,...,15.79,157.94,171.33,45.49114,214.963751,214.97,2149.67,26.668205,25.601858,29.2
3,1333,#A1101856000,NaN,NaN,極洋,2017/02,JP3257200000,1301,2717,2985,...,15.79,157.94,171.33,45.49114,214.963751,214.97,2149.67,26.668205,25.601858,29.2
4,1352,#A1101857000,0,NaN,極洋,2017/03,JP3257200000,1301,2984,3210,...,15.79,157.94,171.33,45.49114,214.963751,214.97,2149.67,26.668205,25.601858,29.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796057,NaN,NaN,NaN,NaN,ベルーナ,1992/08,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60269.230769,130769.230769,NaN,NaN,22346.153846,NaN,NaN
796058,NaN,NaN,NaN,NaN,ベルーナ,1992/09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60269.230769,130769.230769,NaN,NaN,22346.153846,NaN,NaN
796059,NaN,NaN,NaN,NaN,ベルーナ,1992/10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60269.230769,130769.230769,NaN,NaN,22346.153846,NaN,NaN
796060,NaN,NaN,NaN,NaN,ベルーナ,1992/11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60269.230769,130769.230769,NaN,NaN,22346.153846,NaN,NaN


,0,1,2,3,Company,Date,Code,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,2022/09/23 12:48:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,#A1101851000,2017-01-01 00:00:00,NaN,銘柄名,期日,銘柄ｺｰﾄﾞ,流動比率,当座比率,固定比率,...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比
2,1332,#AA914471000,2022-06-01 00:00:00,NaN,極洋,2017/01,1301,172.464797,70.669769,107.099832,...,1.21075,1.047497,1.100407,1.157072,1.428912,15.751563,75260.97,258,28194.97,-9.154919
3,1333,#AA914472000,NaN,NaN,極洋,2017/02,1301,172.464797,70.669769,107.099832,...,1.21075,1.047497,1.100407,1.157072,1.428912,15.751563,75260.97,258,28194.97,-9.154919
4,1352,#AA914473000,0,NaN,極洋,2017/03,1301,172.464797,70.669769,107.099832,...,1.21075,1.047497,1.100407,1.157072,1.428912,15.751563,75260.97,258,28194.97,-9.154919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796057,NaN,NaN,NaN,NaN,ベルーナ,1992/08,NaN,96.976119,77.284212,241.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796058,NaN,NaN,NaN,NaN,ベルーナ,1992/09,NaN,96.976119,77.284212,241.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796059,NaN,NaN,NaN,NaN,ベルーナ,1992/10,NaN,96.976119,77.284212,241.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796060,NaN,NaN,NaN,NaN,ベルーナ,1992/11,NaN,96.976119,77.284212,241.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Company,Date,Code,7,8
0,NaN,NaN,NaN,NaN,NaN
1,銘柄名,期日,銘柄ｺｰﾄﾞ,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）
2,極洋,1992/01,1301,80996630000,7150
3,極洋,1992/02,1301,81563040000,7200
4,極洋,1992/03,1301,71367660000,6300
...,...,...,...,...,...
796047,ベルーナ,2022/02,9997,71377096000,734
796048,ベルーナ,2022/03,9997,70599144000,726
796049,ベルーナ,2022/04,9997,71085364000,731
796050,ベルーナ,2022/05,9997,64861748000,667


# 4. データの整理

## 4.1 企業ごと日付順に並べ替える

本来ならば企業名でもいいはずだが、'コナミHD'と'コナミG'のように同一の企業であるにも関わらず、後々の処理で異なる企業と判断されかねないので、銘柄コードを用いてソートする.

In [7]:
data_0 = data_0.sort_values(by=['Code', 'Date']) #銘柄コードと日付によってソートする
data_1_1 = data_0.dropna(subset=['Code'])  #銘柄コードがNaNである行を全て削除(上場廃止や途中上場の銘柄を除く)

zaimudata_0 = zaimudata_0.sort_values(by=['Code', 'Date']) #銘柄コードと日付によってソートする
zaimudata_1_1 = zaimudata_0.dropna(subset=['Code'])  #銘柄コードがNaNである行を全て削除(上場廃止や途中上場の銘柄を除く)

jikasougakudata_0_3 = jikasougakudata_0_2.sort_values(by=['Code', 'Date']) #銘柄コードと日付によってソートする
jikasougakudata_1_1 = jikasougakudata_0_3.dropna(subset=['Code'])  #銘柄コードがNaNである行を全て削除(上場廃止や途中上場の銘柄を除く)

display(data_1_1, zaimudata_1_1, jikasougakudata_1_1)

,0,1,2,3,Company,Date,6,Code,8,9,...,121,122,123,124,125,126,127,128,129,130
769962,1332,#A1101851000,1992-12-01 00:00:00,NaN,極洋,1992/01,NaN,1301,800,800,...,NaN,NaN,NaN,48.948641,83.261242,NaN,NaN,9.60435,NaN,NaN
769963,1333,#A1101856000,NaN,NaN,極洋,1992/02,NaN,1301,735,740,...,NaN,NaN,NaN,48.948641,83.261242,NaN,NaN,9.60435,NaN,NaN
769964,1352,#A1101857000,0,NaN,極洋,1992/03,NaN,1301,720,733,...,NaN,NaN,NaN,48.948641,83.261242,NaN,NaN,9.60435,NaN,NaN
769965,1375,#A1101858000,NaN,NaN,極洋,1992/04,NaN,1301,625,630,...,NaN,NaN,NaN,48.948641,83.261242,NaN,NaN,9.60435,NaN,NaN
769966,1376,#A1101859000,NaN,NaN,極洋,1992/05,NaN,1301,502,570,...,NaN,NaN,NaN,48.948641,83.261242,NaN,NaN,9.60435,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143553,1301,#A1101868000,2011-01-01 00:00:00,NaN,銘柄名,期日,ISINｺｰﾄﾞ,銘柄ｺｰﾄﾞ,始値,高値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)
300155,1301,#A1101868000,2005-01-01 00:00:00,NaN,銘柄名,期日,ISINｺｰﾄﾞ,銘柄ｺｰﾄﾞ,始値,高値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)
456757,1301,#A1101868000,1999-01-01 00:00:00,NaN,銘柄名,期日,ISINｺｰﾄﾞ,銘柄ｺｰﾄﾞ,始値,高値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)
613359,1301,#A1101868000,1993-01-01 00:00:00,NaN,銘柄名,期日,ISINｺｰﾄﾞ,銘柄ｺｰﾄﾞ,始値,高値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)


,0,1,2,3,Company,Date,Code,7,8,9,...,96,97,98,99,100,101,102,103,104,105
769962,1332,#AA914471000,1992-12-01 00:00:00,NaN,極洋,1992/01,1301,104.893113,70.141352,180.195081,...,1.066531,1.059011,1.083784,0.95355,0.599684,25.601304,141959.232,976,110563.232,-25.496183
769963,1333,#AA914472000,NaN,NaN,極洋,1992/02,1301,104.893113,70.141352,180.195081,...,1.066531,1.059011,1.083784,0.95355,0.599684,25.601304,141959.232,976,110563.232,-25.496183
769964,1352,#AA914473000,0,NaN,極洋,1992/03,1301,104.893113,70.141352,180.195081,...,1.066531,1.059011,1.083784,0.95355,0.599684,25.601304,141959.232,976,110563.232,-25.496183
769965,1375,#AA914474000,NaN,NaN,極洋,1992/04,1301,104.893113,70.141352,180.195081,...,1.066531,1.059011,1.083784,0.95355,0.599684,25.601304,141959.232,976,110563.232,-25.496183
769966,1376,#AA914475000,NaN,NaN,極洋,1992/05,1301,104.893113,70.141352,180.195081,...,1.066531,1.059011,1.083784,0.95355,0.599684,25.601304,141959.232,976,110563.232,-25.496183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143553,1301,#A1101851000,2011-01-01 00:00:00,NaN,銘柄名,期日,銘柄ｺｰﾄﾞ,流動比率,当座比率,固定比率,...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比
300155,1301,#A1101851000,2005-01-01 00:00:00,NaN,銘柄名,期日,銘柄ｺｰﾄﾞ,流動比率,当座比率,固定比率,...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比
456757,1301,#A1101851000,1999-01-01 00:00:00,NaN,銘柄名,期日,銘柄ｺｰﾄﾞ,流動比率,当座比率,固定比率,...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比
613359,1301,#A1101851000,1993-01-01 00:00:00,NaN,銘柄名,期日,銘柄ｺｰﾄﾞ,流動比率,当座比率,固定比率,...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比


,Company,Date,Code,7,8
2,極洋,1992/01,1301,80996630000,7150
3,極洋,1992/02,1301,81563040000,7200
4,極洋,1992/03,1301,71367660000,6300
5,極洋,1992/04,1301,56867564000,5020
6,極洋,1992/05,1301,62305100000,5500
...,...,...,...,...,...
796048,ベルーナ,2022/03,9997,70599144000,726
796049,ベルーナ,2022/04,9997,71085364000,731
796050,ベルーナ,2022/05,9997,64861748000,667
796051,ベルーナ,2022/06,9997,72446780000,745


## 4.2 1992〜1994/11と2022年のデータを削除

In [8]:
index_list_0_1 = data_1_1.query('Date.str.contains("2022")', engine='python') #2022のデータを抽出
index_list_0_2 = data_1_1.query('Date.str.contains("1992")', engine='python') #1992のデータを抽出
index_list_0_3 = data_1_1.query('Date.str.contains("1993")', engine='python') #1993のデータを抽出
index_list_0_4 = data_1_1.query('Date.str.contains("1994")', engine='python') #1994のデータを抽出
index_list_0_5 = index_list_0_1.index.values.tolist() #2022のデータのindexをlist型に変更
index_list_0_6 = index_list_0_2.index.values.tolist() #1992のデータのindexをlist型に変更
index_list_0_7 = index_list_0_3.index.values.tolist() #1993のデータのindexをlist型に変更
index_list_0_8 = index_list_0_4.index.values.tolist() #1994のデータのindexをlist型に変更
data_1_1_0 = data_1_1[~data_1_1.index.isin(index_list_0_5)] #2022のデータのindexにあうものを削除
data_1_1_1 = data_1_1_0[~data_1_1_0.index.isin(index_list_0_6)] #1992のデータのindexにあうものを削除
data_1_1_2 = data_1_1_1[~data_1_1_1.index.isin(index_list_0_7)] #1993のデータのindexにあうものを削除
data_1_1_3 = data_1_1_2[~data_1_1_2.index.isin(index_list_0_8)] #1994のデータのindexにあうものを削除
data_1_1_4 = data_1_1.query('Date.str.contains("1994/12")', engine='python') #1994/12のデータを抽出
data_1_1_5 = pd.concat([data_1_1_4, data_1_1_3]) #1994/12と1995~2021のデータを結合
data_1_1_6 = data_1_1_5.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除)
data_1_1_7 = data_1_1_6.sort_values(by=['Code', 'Date']) #銘柄コードと日付によってソートする

In [9]:
zaimuindex_list_0_1 = zaimudata_1_1.query('Date.str.contains("2022")', engine='python') #2022のデータを抽出
zaimuindex_list_0_2 = zaimudata_1_1.query('Date.str.contains("1992")', engine='python') #1992のデータを抽出zaimu
zaimuindex_list_0_3 = zaimudata_1_1.query('Date.str.contains("1993")', engine='python') #1993のデータを抽出
zaimuindex_list_0_4 = zaimudata_1_1.query('Date.str.contains("1994")', engine='python') #1994のデータを抽出
zaimuindex_list_0_5 = zaimuindex_list_0_1.index.values.tolist() #2022のデータのindexをlist型に変更
zaimuindex_list_0_6 = zaimuindex_list_0_2.index.values.tolist() #1992のデータのindexをlist型に変更
zaimuindex_list_0_7 = zaimuindex_list_0_3.index.values.tolist() #1993のデータのindexをlist型に変更
zaimuindex_list_0_8 = zaimuindex_list_0_4.index.values.tolist() #1994のデータのindexをlist型に変更
zaimudata_1_1_0 = zaimudata_1_1[~zaimudata_1_1.index.isin(zaimuindex_list_0_5)] #2022のデータのindexにあうものを削除
zaimudata_1_1_1 = zaimudata_1_1_0[~zaimudata_1_1_0.index.isin(zaimuindex_list_0_6)] #1992のデータのindexにあうものを削除
zaimudata_1_1_2 = zaimudata_1_1_1[~zaimudata_1_1_1.index.isin(zaimuindex_list_0_7)] #1993のデータのindexにあうものを削除
zaimudata_1_1_3 = zaimudata_1_1_2[~zaimudata_1_1_2.index.isin(zaimuindex_list_0_8)] #1994のデータのindexにあうものを削除
zaimudata_1_1_4 = zaimudata_1_1.query('Date.str.contains("1994/12")', engine='python') #1994/12のデータを抽出
zaimudata_1_1_5 = pd.concat([zaimudata_1_1_4, zaimudata_1_1_3]) #1994/12と1995~2021のデータを結合
zaimudata_1_1_6 = zaimudata_1_1_5.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除)
zaimudata_1_1_7 = zaimudata_1_1_6.sort_values(by=['Code', 'Date']) #銘柄コードと日付によってソートする

In [10]:
jikasougakuindex_list_0_1 = jikasougakudata_1_1.query('Date.str.contains("2022")', engine='python') #2022のデータを抽出
jikasougakuindex_list_0_2 = jikasougakudata_1_1.query('Date.str.contains("1992")', engine='python') #1992のデータを抽出jikasougaku
jikasougakuindex_list_0_3 = jikasougakudata_1_1.query('Date.str.contains("1993")', engine='python') #1993のデータを抽出
jikasougakuindex_list_0_4 = jikasougakudata_1_1.query('Date.str.contains("1994")', engine='python') #1994のデータを抽出
jikasougakuindex_list_0_5 = jikasougakuindex_list_0_1.index.values.tolist() #2022のデータのindexをlist型に変更
jikasougakuindex_list_0_6 = jikasougakuindex_list_0_2.index.values.tolist() #1992のデータのindexをlist型に変更
jikasougakuindex_list_0_7 = jikasougakuindex_list_0_3.index.values.tolist() #1993のデータのindexをlist型に変更
jikasougakuindex_list_0_8 = jikasougakuindex_list_0_4.index.values.tolist() #1994のデータのindexをlist型に変更
jikasougakudata_1_1_0 = jikasougakudata_1_1[~jikasougakudata_1_1.index.isin(jikasougakuindex_list_0_5)] #2022のデータのindexにあうものを削除
jikasougakudata_1_1_1 = jikasougakudata_1_1_0[~jikasougakudata_1_1_0.index.isin(jikasougakuindex_list_0_6)] #1992のデータのindexにあうものを削除
jikasougakudata_1_1_2 = jikasougakudata_1_1_1[~jikasougakudata_1_1_1.index.isin(jikasougakuindex_list_0_7)] #1993のデータのindexにあうものを削除
jikasougakudata_1_1_3 = jikasougakudata_1_1_2[~jikasougakudata_1_1_2.index.isin(jikasougakuindex_list_0_8)] #1994のデータのindexにあうものを削除
jikasougakudata_1_1_4 = jikasougakudata_1_1.query('Date.str.contains("1994/12")', engine='python') #1994/12のデータを抽出
jikasougakudata_1_1_5 = pd.concat([jikasougakudata_1_1_4, jikasougakudata_1_1_3]) #1994/12と1995~2021のデータを結合
jikasougakudata_1_1_6 = jikasougakudata_1_1_5.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除)
jikasougakudata_1_1_7 = jikasougakudata_1_1_6.sort_values(by=['Code', 'Date']) #銘柄コードと日付によってソートする

## 4.3 上場廃止や途中上場のデータを取り除く

In [11]:
#data_1_1_7 = data_0.dropna(subset=['Code'])  #銘柄コードがNaNである行を全て削除(上場廃止や途中上場の銘柄を除く)

name_list_0_1 = data_1_1_7.Code.to_list() #codeがNaNである行を削除した後のデータフレームにおける銘柄コードをリスト化

name_list_0_2 = collections.Counter(name_list_0_1) #銘柄コードが何個重複しているかを確認する

name_list_0_3 = name_list_0_2.most_common() #list型で情報を抜き出す

name_frame_0_1 = pd.DataFrame(name_list_0_3) #name_list_0_3をデータフレームにする

all_data_num_1 = name_frame_0_1[1].max() #上場廃止や途中上場がない企業のデータ数(またはカウント数、つまり何ヶ月分か)を取得

name_frame_0_2 = name_frame_0_1[name_frame_0_1[1] == all_data_num_1] #上場廃止や途中上場がない企業の銘柄コードのみを抽出する

#name_frame_0_2.shape[0] #上場廃止や途中上場がない企業の数(つまり分析に使える企業数)

name_list_0_4 = name_frame_0_2[0].values.tolist() #上場廃止や途中上場がない企業の銘柄コードの名称リストを作成

data_1_2 = data_1_1_7[data_1_1_7['Code'].isin(name_list_0_4)] #全ての時点で存在し続けていない銘柄を削除

data_1_3 = data_1_2.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除)

data_1_3 = data_1_3.drop(data_1_3.columns[[0, 1, 2, 3]], axis=1) #余分な列を削除

name_list_0_5 = data_0_1.iloc[1,4:].values.tolist() #カラム名に設定するものをリストとして抽出する

data_1_3 = data_1_3.set_axis(name_list_0_5 , axis=1) #カラム名を設定

data_1_3 = data_1_3.rename(columns={'銘柄ｺｰﾄﾞ': '銘柄コード'}) #特定の列の名前を変更

In [12]:
#zaimudata_1_1_7 = zaimudata_0.dropna(subset=['Code'])  #銘柄コードがNaNである行を全て削除(上場廃止や途中上場の銘柄を除く)

zaimuname_list_0_1 = zaimudata_1_1_7.Code.to_list() #codeがNaNである行を削除した後のデータフレームにおける銘柄コードをリスト化

zaimuname_list_0_2 = collections.Counter(zaimuname_list_0_1) #銘柄コードが何個重複しているかを確認する

zaimuname_list_0_3 = zaimuname_list_0_2.most_common() #list型で情報を抜き出す

zaimuname_frame_0_1 = pd.DataFrame(zaimuname_list_0_3) #name_list_0_3をデータフレームにする

zaimuall_data_num_1 = zaimuname_frame_0_1[1].max() #上場廃止や途中上場がない企業のデータ数(またはカウント数、つまり何ヶ月分か)を取得

zaimuname_frame_0_2 = zaimuname_frame_0_1[zaimuname_frame_0_1[1] == zaimuall_data_num_1] #上場廃止や途中上場がない企業の銘柄コードのみを抽出する

#zaimuname_frame_0_2.shape[0] #上場廃止や途中上場がない企業の数(つまり分析に使える企業数)

zaimuname_list_0_4 = zaimuname_frame_0_2[0].values.tolist() #上場廃止や途中上場がない企業の銘柄コードの名称リストを作成

zaimudata_1_2 = zaimudata_1_1_7[zaimudata_1_1_7['Code'].isin(zaimuname_list_0_4)] #全ての時点で存在し続けていない銘柄を削除

zaimudata_1_3 = zaimudata_1_2.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除)

zaimudata_1_3 = zaimudata_1_3.drop(zaimudata_1_3.columns[[0, 1, 2, 3]], axis=1) #余分な列を削除

zaimuname_list_0_5 = zaimudata_0_1.iloc[1,4:].values.tolist() #カラム名に設定するものをリストとして抽出する

zaimudata_1_3 = zaimudata_1_3.set_axis(zaimuname_list_0_5 , axis=1) #カラム名を設定

zaimudata_1_3 = zaimudata_1_3.rename(columns={'銘柄ｺｰﾄﾞ': '銘柄コード'}) #特定の列の名前を変更

In [13]:
#jikasougakudata_1_1_7 = jikasougakudata_0.dropna(subset=['Code'])  #銘柄コードがNaNである行を全て削除(上場廃止や途中上場の銘柄を除く)

jikasougakuname_list_0_1 = jikasougakudata_1_1_7.Code.to_list() #codeがNaNである行を削除した後のデータフレームにおける銘柄コードをリスト化

jikasougakuname_list_0_2 = collections.Counter(jikasougakuname_list_0_1) #銘柄コードが何個重複しているかを確認する

jikasougakuname_list_0_3 = jikasougakuname_list_0_2.most_common() #list型で情報を抜き出す

jikasougakuname_frame_0_1 = pd.DataFrame(jikasougakuname_list_0_3) #name_list_0_3をデータフレームにする

jikasougakuall_data_num_1 = jikasougakuname_frame_0_1[1].max() #上場廃止や途中上場がない企業のデータ数(またはカウント数、つまり何ヶ月分か)を取得

jikasougakuname_frame_0_2 = jikasougakuname_frame_0_1[jikasougakuname_frame_0_1[1] == jikasougakuall_data_num_1] #上場廃止や途中上場がない企業の銘柄コードのみを抽出する

#jikasougakuname_frame_0_2.shape[0] #上場廃止や途中上場がない企業の数(つまり分析に使える企業数)

jikasougakuname_list_0_4 = jikasougakuname_frame_0_2[0].values.tolist() #上場廃止や途中上場がない企業の銘柄コードの名称リストを作成

jikasougakudata_1_2 = jikasougakudata_1_1_7[jikasougakudata_1_1_7['Code'].isin(jikasougakuname_list_0_4)] #全ての時点で存在し続けていない銘柄を削除

jikasougakudata_1_3 = jikasougakudata_1_2.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除)

jikasougakuname_list_0_5 = jikasougakudata_0_1.iloc[1,[4,5,6,7,8]].values.tolist() #カラム名に設定するものをリストとして抽出する

jikasougakudata_1_3 = jikasougakudata_1_3.set_axis(jikasougakuname_list_0_5 , axis=1) #カラム名を設定

jikasougakudata_1_3 = jikasougakudata_1_3.rename(columns={'銘柄ｺｰﾄﾞ': '銘柄コード'}) #特定の列の名前を変更

In [14]:
display(data_1_3, zaimudata_1_3, jikasougakudata_1_3)

,銘柄名,期日,ISINｺｰﾄﾞ,銘柄コード,始値,高値,安値,終値,売買代金,調整済始値,...,1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,配当性向(企業発表)
0,極洋,1994/12,NaN,1301,378,419,371,402,0,3780,...,NaN,NaN,NaN,18.255166,77.134257,NaN,NaN,11.396238,NaN,NaN
1,極洋,1995/01,NaN,1301,407,413,358,386,0,4070,...,NaN,NaN,NaN,18.255166,77.134257,NaN,NaN,11.396238,NaN,NaN
2,極洋,1995/02,NaN,1301,388,400,346,352,0,3880,...,NaN,NaN,NaN,18.255166,77.134257,NaN,NaN,11.396238,NaN,NaN
3,極洋,1995/03,NaN,1301,352,380,335,340,0,3520,...,NaN,NaN,NaN,18.255166,77.134257,NaN,NaN,11.396238,NaN,NaN
4,極洋,1995/04,NaN,1301,330,370,315,355,0,3300,...,NaN,NaN,NaN,18.255166,77.134257,NaN,NaN,11.396238,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,JP3835650007,9997,883,900,773,822,6024130900,883,...,114.17,NaN,NaN,210.972162,1164.962737,1164.97,NaN,133.975702,214.882502,14.5
370821,ベルーナ,2021/09,JP3835650007,9997,822,892,819,854,6967618600,822,...,114.17,NaN,NaN,210.972162,1164.962737,1164.97,NaN,133.975702,214.882502,14.5
370822,ベルーナ,2021/10,JP3835650007,9997,848,851,792,798,3203755900,848,...,114.17,NaN,NaN,210.972162,1164.962737,1164.97,NaN,133.975702,214.882502,14.5
370823,ベルーナ,2021/11,JP3835650007,9997,793,795,665,667,4021996800,793,...,114.17,NaN,NaN,210.972162,1164.962737,1164.97,NaN,133.975702,214.882502,14.5


,銘柄名,期日,銘柄コード,流動比率,当座比率,固定比率,固定長期適合率,自己資本比率,負債比率,他人資本ｺｽﾄ(有利子負債金利),...,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比
0,極洋,1994/12,1301,103.526052,68.368994,156.351568,88.910582,13.718933,628.919661,4.183892,...,1.057022,1.049759,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633
1,極洋,1995/01,1301,103.526052,68.368994,156.351568,88.910582,13.718933,628.919661,4.183892,...,1.057022,1.049759,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633
2,極洋,1995/02,1301,103.526052,68.368994,156.351568,88.910582,13.718933,628.919661,4.183892,...,1.057022,1.049759,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633
3,極洋,1995/03,1301,103.526052,68.368994,156.351568,88.910582,13.718933,628.919661,4.183892,...,1.057022,1.049759,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633
4,極洋,1995/04,1301,103.526052,68.368994,156.351568,88.910582,13.718933,628.919661,4.183892,...,1.057022,1.049759,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997,207.626921,122.372181,110.705914,67.776183,46.883781,112.750844,0.264562,...,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466
370821,ベルーナ,2021/09,9997,207.626921,122.372181,110.705914,67.776183,46.883781,112.750844,0.264562,...,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466
370822,ベルーナ,2021/10,9997,207.626921,122.372181,110.705914,67.776183,46.883781,112.750844,0.264562,...,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466
370823,ベルーナ,2021/11,9997,207.626921,122.372181,110.705914,67.776183,46.883781,112.750844,0.264562,...,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466


,銘柄名,期日,銘柄コード,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）
0,極洋,1994/12,1301,45539364000,4020
1,極洋,1995/01,1301,43726852000,3860
2,極洋,1995/02,1301,39875264000,3520
3,極洋,1995/03,1301,38515880000,3400
4,極洋,1995/04,1301,40215110000,3550
...,...,...,...,...,...
370820,ベルーナ,2021/08,9997,79934568000,822
370821,ベルーナ,2021/09,9997,83046376000,854
370822,ベルーナ,2021/10,9997,77600712000,798
370823,ベルーナ,2021/11,9997,64861748000,667


## 4.4 追加データと最初に読み込んだデータの銘柄の整合性の確認

In [15]:
#d1 = data_1_8.iloc[:,0] #銘柄名の列を抽出
d1 = zaimudata_1_3.iloc[:,2] #銘柄名の列を抽出
d2 = data_1_3.iloc[:,3] #銘柄名の列を抽出
d3 = jikasougakudata_1_3.iloc[:,2] #銘柄名の列を抽出
print(d1.equals(d2), d1.equals(d3)) #これがいずれもTrueならば二つのデータフレームにある銘柄を全て等しい

True True


# 5. データの結合

## 5.1 重複データの抽出・削除
最初に取得したデータと追加財務データで重複しているデータを抽出、追加財務データの方は残して、最初に取得したデータの方から削除する。

In [16]:
data_1_4 = pd.DataFrame(data_1_3.columns)
zaimudata_1_4 = pd.DataFrame(zaimudata_1_3.columns)
jikasougakudata_1_4 = pd.DataFrame(jikasougakudata_1_3.columns)
jikasougakudata_1_5 = jikasougakudata_1_3.iloc[:,[3,4]]
checkframe_1 = pd.concat([data_1_4,zaimudata_1_4,jikasougakudata_1_4],axis=0) #確認用データフレームの作成
checkframe_2 = checkframe_1.reset_index().iloc[:,1] #indexの振り直し
checkframe_3 = pd.DataFrame(checkframe_2)
checkframe_4 = checkframe_3[checkframe_3.duplicated()].T #重複しているデータを抽出
display(checkframe_4)
data_1_5 = data_1_3.drop(checkframe_4.iloc[0,:].to_list(), axis=1) #最初に取得したデータから重複データの列を削除

,127,128,129,130,131,132,133,134,135,136,...,173,176,191,192,193,194,196,229,230,231
0,銘柄名,期日,銘柄コード,流動比率,当座比率,固定比率,固定長期適合率,自己資本比率,負債比率,他人資本ｺｽﾄ(有利子負債金利),...,使用資本利益率(ROCE),総資産回転率,売上高伸び率,営業利益伸び率,経常利益伸び率,当期利益伸び率,配当性向(企業発表),銘柄名,期日,銘柄コード


## 5.2 データの結合

In [17]:
bondeddata_1 = pd.concat([data_1_5, zaimudata_1_3,jikasougakudata_1_5], axis=1) #重複データを削除した最初に取得したデータと追加財務データを結合
bondeddata_2 = bondeddata_1.drop(bondeddata_1[['銘柄名','期日','銘柄コード']], axis=1) #銘柄名,'期日','銘柄コード'の列を削除
bondeddata_3 = pd.concat([bondeddata_1[['銘柄名','期日','銘柄コード']], bondeddata_2], axis=1) #銘柄名,'期日','銘柄コード'の列を挿入
display(bondeddata_3)

,銘柄名,期日,銘柄コード,ISINｺｰﾄﾞ,始値,高値,安値,終値,売買代金,調整済始値,...,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）
0,極洋,1994/12,1301,NaN,378,419,371,402,0,3780,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,45539364000,4020
1,極洋,1995/01,1301,NaN,407,413,358,386,0,4070,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,43726852000,3860
2,極洋,1995/02,1301,NaN,388,400,346,352,0,3880,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,39875264000,3520
3,極洋,1995/03,1301,NaN,352,380,335,340,0,3520,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,38515880000,3400
4,極洋,1995/04,1301,NaN,330,370,315,355,0,3300,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,40215110000,3550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997,JP3835650007,883,900,773,822,6024130900,883,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,79934568000,822
370821,ベルーナ,2021/09,9997,JP3835650007,822,892,819,854,6967618600,822,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,83046376000,854
370822,ベルーナ,2021/10,9997,JP3835650007,848,851,792,798,3203755900,848,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,77600712000,798
370823,ベルーナ,2021/11,9997,JP3835650007,793,795,665,667,4021996800,793,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,64861748000,667


In [18]:
bondeddata_3.銘柄コード.nunique(dropna = True) #使用可能な銘柄数
bondeddata_3.期日.nunique(dropna = True) #期日数
print('使用可能な企業数は',bondeddata_3.銘柄コード.nunique(dropna = True),'、', '期日数は',bondeddata_3.期日.nunique(dropna = True),'である。')

使用可能な企業数は 1141 、 期日数は 325 である。


# 6. 準備後データの書き出し(pickleによる高速化)

In [19]:
bondeddata_3.to_pickle('./organized_data.pickle') #ファイルの書き出し

In [20]:
print(os.path.getsize('./organized_data.pickle')) #ファイルの大きさを表示

562083834
